# Alpha Santorini zero v01 
This notebook is a first attempt to implement the methods described in https://web.stanford.edu/~surag/posts/alphazero.html for Santorini. Since for small values of <i>board_dim</i> and <i>units_per_player</i>, an optimal policy $\pi^*$ can easily be determined, we test whether the NN can compete with $\pi^*$. Note that $\pi^*$ might not be unique, but its performance is.

### Choosing the dimensions of the Environment 

In [427]:
import santorini as san
import numpy as np
from tqdm import tqdm_notebook

env = san.Environment(board_dim=2, units_per_player=1)

if env.units_per_player == 1 and env.board_dim == 2: max_actions = 4
else: max_actions = 64*env.units_per_player

start_state_collection = { (2,1) : '00000011', 
                           (3,1) : '0000000000022', (3,2) : '00000000000022022',
                           (4,1) : '00000000000000000033', (4,2) : '000000000000000000033033',
                           (5,1) : '00000000000000000000000001133', (5,2) : '000000000000000000000000011133133' }                    

s_start = start_state_collection[(env.board_dim, env.units_per_player)]
san.State.from_string(s_start , board_dim=env.board_dim).print()

---P--
P0  0 
 0 O0 
------


### Remarks on Notation
We identify actions of a state with the targetsstate since state-action transitions are deterministic. The symbol $s$ will denote the string-representation of a state, e.g. $s=$'00000011'. We therefore introduce some helper functions for the conversion to the following other representations: 
* an instance of the <i>class santorini.State</i> 
* a python-list of integers 

We also introduce the method <b>children(s)</b> that returns the childnodes of a state $s$ in their string-representation. <b>[!]Note:</b> The order in which they are returned defines the indexing of states, in particular for policy-vectors!

In [385]:
def state(s): return san.State.from_string(s, board_dim=env.board_dim)

def children(s): return [env.do_play(p, state(s)).string() for p in env.get_plays(state(s))]

def integers(s): return [int(c) for c in s]

### Computing the optimal policy

In [428]:
import alphabeta
negamax = alphabeta.alphabeta(env.score, env.get_plays, env.do_play)

In [442]:
# get the maximum depth
t_max=0
for t in range(0, 4*env.board_dim*env.board_dim):
    if negamax.basic(state(s_start), t_max)[0] == 0: t_max+=1
    else: break

# populate keys: all non-terminal states
pi_opt = { st.string() : 0 for st in state('00000011').equiv_class()}
for t in range(0, t_max):
    for s in list(pi_opt.keys()):  
        for c in children(s):
            if env.score(state(c)) == 0: pi_opt.update({c : 0})
# populate values    
for s in list(pi_opt.keys()): 
    v, play = negamax.basic(state(s), t_max)
    pi_opt.update({s : [env.do_play(play, state(s)).string(), v]})

# reformat output to match NN architecture
def pi_opt_predict(s):
    ss, v = pi_opt[s]
    return [[1 if i==children(s).index(ss) else 0 for i in range(0,max_actions) ], v]


## Generating training data
The data for the training of the NN is gathered via self-play. A tree search (TS) traverses the game tree node by node by the following rules, starting from a root node: When a terminal node is encountered, its value ($\pm1$) is returned and the TS terminates. When a so-far unvisited node $s$ is encountered (instead of a random rollout as a MCTS would do) the expected reward of the node is returned by calling the NN on $s$. Moreover, the TS ends in this case with $Q(s,\cdot)$ and $N(s,\cdot)$ initialised to zero and $P(s, \cdot) \leftarrow \text{NN}(s)$. If the (non-terminal) node $s$ has been visited, then the search continues on the next node $s^\prime$ chosen as the maximizer of the upper confidence bound $U(s,s^\prime):=Q(s,s\prime) + c_{UCB}P(s,s^\prime)\frac{\sqrt{\sum_{s^\prime} N(s,s^\prime)}}{N(s,s^\prime)+1}$. Whenever the TS ends (unvisited or terminal node), the N and Q-values along the serach path are updated: Given an edge $ss^\prime$, we increment $N(s,s^\prime)$ by one and then apply the update rule $Q(s,s^\prime) \leftarrow \frac{v-Q(s,s^\prime)}{N(s,s^\prime)}$.  

Finally, after a fixed number of simulations (searchs), the TS returns an estaimte of the policy at the root node, which is $\pi(s_{root},s^\prime) \propto N(s,s^\prime)^\tau $ for a given temperatur $\tau$.

#### Algorithm: self_play($N_{TS}$):
1. Initialise $s=s_{init}$,  a tree $T$ and a game log $L$.  
2. While $s$ is not terminal, repeat:     
    a) $N \leftarrow$ TS($s,N_{TS}$)  
    b) If $\tau$ is given, $\pi[s][s^\prime] :=   (N[s][s^\prime])^\tau /( \sum_{s^\prime} (N[s][s^\prime])^\tau )$, else $\pi[s][s^\prime] := \delta(s^\prime - \arg\max N[s][\cdot])$.  
    c) Add $(s,\pi[s], \text{None} )$ to $L$.  
    d) Sample the next $s$ from its children (stored in $T$) with probability $\pi[s]$  .
4. Propagate the game rewards (update the 3rd value for each $l\in L$), depending on who won.
5. <b>Return</b> $L$.

#### Algorithm TS($s,N_{TS}$): 
1. Initialise $N,Q,P$ and the tree $T$.  
2. Run the subroutine TS_run($s$) $N_{TS}$-times  
3. <b>Return</b> $N$.  

Subroutine TS_run($s$):
1. If $s$ is terminal <b>return</b> -score($s$).
2. If $s \not \in T$ then add it to $T$ (compute and store the children). 
3. If $s$ has not been visited yet:  
    a) Initialise $Q[s][\cdot ] = N[s][\cdot] = 0$.  
    b) Get $P[s][\cdot], v[s] \leftarrow NN[s]$.   
    c) <b>Return</b> $-v$.
4. Obtain $s^\prime \leftarrow \arg \max_{s^\prime}U[s][s^\prime]$.
5. Do a recursive call $v \leftarrow$  <b>TS_run</b>($s^\prime$).
6. update first $N[s][s^\prime]\;+\!=1$ then $Q[s][s^\prime]\;+\!= \frac{v-Q[s][s^\prime]}{N[s][s^\prime]}$.
7. <b>Return</b> $-v$.
    
Note that TS_run returns the negative value, since a value is always from the <i>active</i> players perspective, which for the calling level (we have a recursive method) is the opponent.

In [444]:
def TS(s, NN_predict_fct, N_searches=100, UCB_cst=1.0, tree=None):
    if tree==None: tree = {} 
    N, Q, P = {}, {}, {}
    
    def TS_run(s):
        if env.score(state(s)) != 0: return -env.score(state(s))
        if s not in tree.keys(): tree.update({s : children(s)})
        if s not in N.keys():
            Q[s], N[s] = {ss : 0 for ss in tree[s]}, {ss : 0 for ss in tree[s]}
            P[s], val = NN_predict_fct(s)
            return -val            
        U = [Q[s][ss] + UCB_cst*P[s][tree[s].index(ss)]*np.sqrt(sum(N[s].values())/(N[s][ss] +1))  for ss in tree[s]]
        ss = tree[s][np.argmax(U)]
        val = TS_run(ss)
        N[s][ss]+=1
        Q[s][ss]+= (val - Q[s][ss])/N[s][ss]
        return -val
    
    for n in range(0, N_searches): TS_run(s)
    return N

def self_play(NN_predict_fct, temp=None, N_treesearches=100, UCB_cst=1.0):
    """
    temp = None: probability vectors pi[s] are deterministic, else choose temp > 0
    """
    s = s_start
    tree={}    
    game_log = []
    while env.score(state(s)) == 0:
        N = TS(s, NN_predict_fct, N_searches=N_treesearches, UCB_cst=UCB_cst, tree=tree)
        if temp == None:
            pi = [1 if i == np.argmax(N[s]) else 0 for i in range(0,max_actions)]
        else:
            pi = [pow(N[s][ss], temp)/sum([pow(N[s][ss], temp) for ss in tree[s]])  for ss in tree[s]]
            for a in range(0, max_actions - tree[s].__len__()): pi.append(0)
        game_log += [[s, pi, None]]
        s = np.random.choice(tree[s], p=pi[:tree[s].__len__()]) 
    
    L = game_log.__len__()
    reward = env.score(state(s))
    for i in range(L-2, -1, -2): game_log[i][2] = reward
    for i in range(L-1, -1, -2): game_log[i][2] = -1*reward
    return game_log

## Verifying the TS algorithm

In [471]:
def pi_rand_predict(s):    
    C = children(s)
    N_c = C.__len__() 
    policy = [1/N_c if i < N_c else 0 for i in range(0, max_actions)]
    return [policy, 0]        

def TS_test(s, NN_predict_fct, N_searches=1000, UCB_cst=1.0):
    stats = TS(s, NN_predict_fct, N_searches=N_searches, UCB_cst=UCB_cst, tree=None)[s]
    state(s).print()
    print('* TS proabilities for children by visit count:')
    for c in children(s):
        state(c).print()
        print( str(round(100*stats[c]/sum(stats.values()), 1)),'%')
        print('')


def compare(predict_fct_1, predict_fct_2, N_games=500):
    '''
    Returns the win-percentage of predict_fct_1. Starting players alternate.
    '''
    N_wins = 0
    for n in tqdm_notebook(range(0,N_games), desc='test games', leave=False):
        s = s_start
        turns_played = 0
        while env.score(state(s)) == 0:
            C = children(s)
            if turns_played%2 == n%2: pi = predict_fct_1(s)[0][:C.__len__()]
            else: pi = predict_fct_2(s)[0][:C.__len__()]
            s = np.random.choice(C, p = [ p/(sum(pi)) for p in pi])
            turns_played+=1            
        N_wins += int(turns_played%2 == n%2 and env.score(state(s)) == 1)\
                    +int(turns_played%2 != n%2 and env.score(state(s)) == -1)          
    return N_wins/N_games

In [463]:
compare(pi_opt_predict, pi_rand_predict)

0.928

In [470]:
TS_test('23220010', pi_rand_predict, N_searches=1000, UCB_cst=1.0)

---P--
P2  3 
O2  2 
------
* TS proabilities for children by visit count:
---P--
 3 O3 
P2  2 
------
33.3 %

---P--
 2 O3 
P2  3 
------
33.2 %

---P--
 3  3 
P2 O2 
------
1.1 %

---P--
 2  4 
P2 O2 
------
32.3 %



## The NN and its policy iteration algorithm

In [524]:
import h5py
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import*

class NeuralNet():
    def __init__(self, env):
        input_dim = env.board_dim*env.board_dim+4*env.units_per_player
        learning_rate = 0.001

        input_states = keras.Input(shape=(input_dim,))
        layer1 = Dense(64, activation='softmax')(input_states)
        layer2 = Dense(256, activation='softmax')(layer1)
        layer3 = Dense(128, activation='softmax')(layer2)
        layer4 = Dense(64, activation='softmax')(layer3)
        pi = Dense(max_actions, activation='softmax', name='pi')(layer4)  
        v = Dense(1, activation='sigmoid', name='v')(layer1)                    
        self.model = keras.models.Model(inputs=input_states, outputs=[pi, v])
        self.model.compile(loss=['categorical_crossentropy','mean_squared_error'], 
                           optimizer=keras.optimizers.Adam(learning_rate))
        
    def info(self):
        self.model.summary()
        
    def train(self, examples, epochs=100, verbose=False):
        s, pi, v = [], [], []
        for ex in examples:
            s.append(integers(ex[0]))
            pi.append(np.array(ex[1]))
            v.append(ex[2])
        x, y = np.array(s), [np.array(pi), np.array(v)]
        self.model.fit(x, y, epochs=epochs, verbose=int(verbose))
    
    def predict(self, s):
        pi, v = self.model.predict(np.array([integers(s)]))
        return pi[0], v[0]
    
    def detailed_predict(self, s):
        p, v = self.predict(s) 
        C = children(s)
        state(s).print()
        print('* v:',v[0])
        print('* NN predictions for children:')
        for i in range(0, C.__len__()):
            state(C[i]).print()
            print(p[i] ,'%')
            print('')
    
    def play_vs(self, other, N_games=500):
        return compare(self.predict, other.predict, N_games=N_games)     
    
    def save(self, name):
        self.model.save_weights(name)
    
    def load(self, name):
        self.model.load_weights(name)
    
    def self_improve(self, threshold=0.55, train_data_size=100 ,N_treesearches=100, temp=1, UCB_cst=1.0, 
                     train_epochs=100, stagnation_tol=0.03, N_benchmark_games=500):
        self.save('./test_oldNN.h5')
        
        oldNN = NeuralNet(env)
        oldNN.load('./test_oldNN.h5')
        
        win_percentage = [0]
        train_data = [] 
        for _ in tqdm_notebook(range(0, train_data_size), desc='selfplays', leave=False):
            train_data += self_play(self.predict, temp=temp, N_treesearches=N_treesearches, UCB_cst=UCB_cst)
        
        # succesively try to train 10 times, unless stagnation or the goal threshold is reached
        train_iterator = tqdm_notebook(range(0, 10), desc='train round', leave=False)
        for t in train_iterator:
            self.train(train_data, epochs=train_epochs)
            win_percentage += [self.play_vs(oldNN, N_games=N_benchmark_games)]                
            if (t > 0 and win_percentage[-1] - win_percentage[-2] < -stagnation_tol) or win_percentage[-1]>=threshold: 
                train_iterator.close()
                break
        print('win %:',win_percentage[:])
        if  win_percentage[-1]>=threshold: 
            self.save('./test_newNN.h5')
            print('* new weights saved to ./test_newNN.h5')
        else: 
            self.load('./test_oldNN.h5')
            print('* old weights restored')
        
        

In [493]:
nn = NeuralNet(env)
nn.info()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_73 (InputLayer)           (None, 8)            0                                            
__________________________________________________________________________________________________
dense_288 (Dense)               (None, 64)           576         input_73[0][0]                   
__________________________________________________________________________________________________
dense_289 (Dense)               (None, 256)          16640       dense_288[0][0]                  
__________________________________________________________________________________________________
dense_290 (Dense)               (None, 128)          32896       dense_289[0][0]                  
__________________________________________________________________________________________________
dense_291 

### Training

In [494]:
nn.self_improve(threshold=0.55, train_data_size=50, N_treesearches=1000, temp=1, UCB_cst=0.5,
                train_epochs=100, stagnation_tol=0.03, N_benchmark_games=1000)


last win %: [0.478, 0.503, 0.527, 0.561]


We obtained an improvemed NN. Let's pit it against $\pi_{rand}$ and $\pi^*$:

In [495]:
compare(nn.predict, pi_rand_predict, N_games=10000)

0.5534

In [496]:
compare(nn.predict, pi_opt_predict, N_games=10000)

0.2391

In [497]:
compare(pi_rand_predict, pi_opt_predict, N_games=10000)

0.0599

We have a closer look at the predictions:

In [498]:
nn.detailed_predict('23220010')

---P--
P2  3 
O2  2 
------
* v: 0.8320611
* NN predictions for children:
---P--
 3 O3 
P2  2 
------
0.9631542 %

---P--
 2 O3 
P2  3 
------
0.036464352 %

---P--
 3  3 
P2 O2 
------
0.00011417707 %

---P--
 2  4 
P2 O2 
------
0.00026730486 %



We keep self-improving

In [500]:
nn.self_improve(threshold=0.55, train_data_size=50, N_treesearches=1000, temp=1, UCB_cst=0.8,
                train_epochs=100, stagnation_tol=0.03, N_benchmark_games=1000)

last win %: [0.526, 0.533, 0.574]


In [502]:
compare(nn.predict, pi_rand_predict, N_games=10000)

0.6157

In [503]:
compare(nn.predict, pi_opt_predict, N_games=10000)

0.44

In [505]:
nn.save('./44pct_bdim2_upp1.h5')

In [506]:
nn.self_improve(threshold=0.55, train_data_size=50, N_treesearches=1000, temp=1, UCB_cst=0.8,
                train_epochs=100, stagnation_tol=0.03, N_benchmark_games=1000)

last win %: [0.517, 0.536, 0.54, 0.543, 0.544, 0.536, 0.562]


In [507]:
compare(nn.predict, pi_opt_predict, N_games=10000)

0.4979

In [508]:
nn.save('./49pct_bdim2_upp1.h5')

We try to self-improve one more time, lowering the threshold as we are close to $\pi^*$

In [516]:
nn.self_improve(threshold=0.52, train_data_size=100, N_treesearches=1000, temp=1, UCB_cst=1.0,
                train_epochs=100, stagnation_tol=0.03, N_benchmark_games=1000)

last win %: [0.511, 0.523]


In [517]:
compare(nn.predict, pi_opt_predict, N_games=10000)

0.499

In [519]:
nn.save('./49_99pct_bdim2_upp1.h5')

In [520]:
compare(nn.predict, pi_rand_predict, N_games=10000)

0.6904

In [521]:
compare(pi_opt_predict, pi_rand_predict, N_games=10000)

0.9352

# Conclusion
We succesfully trained a NN to win against an optimal policy about 49,9 percent of the games. Note though, that against a random policy, the NN performs significantly worse thatn the optimal policy. This indicates that it found good strategies to beat expert level play, but not random plays...